In [1]:
import random
import gurobipy as gp
import numpy as np
from gurobipy import GRB
from operator import add

In [104]:
# data
D_bl = 10
PV = 5
M = 200

In [105]:
m = gp.Model('test')
# vars
pv = m.addVar(vtype=GRB.INTEGER, name='pv')
pv_sur = m.addVar(vtype=GRB.BINARY, name='pv_sur')
# constr
m.addConstr(pv <= PV, name='pv_ub')
m.addConstr(-(M * pv_sur) + pv <= D_bl, name='pv_sur_t')
m.addConstr(M * (1 - pv_sur) + pv >= D_bl, name='pv_sur_f')
# obj
m.setObjective(pv, GRB.MAXIMIZE)
m.update()
[m.getConstrByName('pv_ub').RHS, m.getConstrByName('pv_sur_t').RHS, m.getConstrByName('pv_sur_f').RHS]

[5.0, 10.0, -190.0]

In [111]:
D_bl_v1 = 10
PV_v1 = 5
m.setAttr('RHS', m.getConstrByName('pv_ub'), PV_v1)
m.setAttr('RHS', m.getConstrByName('pv_sur_t'), D_bl_v1)
m.setAttr('RHS', m.getConstrByName('pv_sur_f'), D_bl_v1 - M)
[m.getConstrByName('pv_sur_t').RHS, m.getConstrByName('pv_sur_f').RHS]

[10.0, -190.0]

In [112]:
# obj
m.optimize()
m.getVars()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x9c9f0238
Variable types: 0 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 2e+02]

Loaded MIP start from previous solve with objective 5

Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


[<gurobi.Var pv (value 5.0)>, <gurobi.Var pv_sur (value 0.0)>]

In [2]:
# generate mock data
def generate_data():
    PV = [random.randrange(10) for i in range(5)]
    D_bl = [random.randrange(6) for i in range(5)]
    H_bl = [i * 2 for i in D_bl]
    price = [1 for i in range (5)]

    return PV, D_bl, H_bl, price

In [3]:
def print_results():
    print(f'D_bl:\t {D_bl}')
    print(f'H_bl:\t {H_bl}')
    print(f'Sum_bl:  {[int(D_bl[i] + H_bl[i]) for i in range(len(D_bl))]}')
    print(f'PV:\t\t {PV}')
    print(f'pv:\t\t', [int(i.X) for i in m.getVars() if 'pv' in i.VarName])
    print(f'g_d:\t', [int(i.X) for i in m.getVars() if 'g' in i.VarName])
    print(f'sum(g_d, pv) =', np.sum([int(i.X) for i in m.getVars() if any(x in i.VarName for x in ('pv', 'g'))]))
    print(f'sum(D_bl, H_bl) =', np.sum(list(map(add, D_bl, H_bl))))
    return

In [4]:
# set up model
m = gp.Model('energy')

# set up variables
time_index = range(5)
pv = m.addVars(time_index, vtype=GRB.INTEGER, name='pv')
g_d = m.addVars(time_index, vtype=GRB.INTEGER, lb=0, name='g_d')

# get mock data
PV, D_bl, H_bl, price = generate_data()

# add constraints
for i in time_index:
    m.addConstr(pv[i] <= PV[i], name=f'pv_ub_{i}')
    m.addConstr(g_d[i] + pv[i] >= D_bl[i] + H_bl[i], name=f'd_bl_{i}')
m.addConstr(gp.quicksum(g_d[i] + pv[i] for i in time_index) >= gp.quicksum(D_bl[i] + H_bl[i] for i in time_index), name='d_bl_sum')

# set objective
m.setObjective(gp.quicksum(g_d[i] * price[i] + pv[i] * 0.01 * price[i] for i in time_index), GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-31


In [5]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 11 rows, 10 columns and 25 nonzeros
Model fingerprint: 0x994315aa
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 4e+01]
Found heuristic solution: objective 36.0000000
Presolve removed 11 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 20.16 36 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.016000000000e+01, best bound 2.016000000000e+01, gap 0.0000%


In [6]:
print_results()

D_bl:	 [1, 1, 5, 2, 3]
H_bl:	 [2, 2, 10, 4, 6]
Sum_bl:  [3, 3, 15, 6, 9]
PV:		 [2, 5, 2, 3, 6]
pv:		 [2, 3, 2, 3, 6]
g_d:	 [1, 0, 13, 3, 3]
sum(g_d, pv) = 36
sum(D_bl, H_bl) = 36


In [ ]:
# feed in different time series with one external variable on RHS
PV = D_bl.copy()
m.setAttr('RHS', [m.getConstrByName(f'pv_ub_{i}') for i in time_index], PV)

In [310]:
# feed in different time series with two external variables on RHS
D_bl = [5,4,3,2,1]
H_bl = [1,2,3,4,5]
m.setAttr('RHS', [m.getConstrByName(f'd_bl_{i}') for i in time_index], [D_bl[i] + H_bl[i] for i in time_index])

In [317]:
# alter summation constraint (one value) in RHS
m.setAttr('RHS', m.getConstrByName('d_bl_sum'), np.sum(list(map(add, D_bl, H_bl))))

In [190]:
# alter objective function -> just execute m.setObjective() one more time
price = [2 for i in range(5)]
m.setObjective(gp.quicksum(g_d[i] * price[i] for i in time_index), GRB.MINIMIZE)
m.optimize()
m.ObjVal

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 10 rows, 10 columns and 15 nonzeros
Model fingerprint: 0x2c3c0261
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]

Loaded MIP start from previous solve with objective 34

Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 34 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.400000000000e+01, best bound 3.400000000000e+01, gap 0.0000%


In [283]:
l1 = [1,2,3] * 96
l2 = [2,4,6] * 96

In [291]:
%timeit list(map(add, l1, l2))
%timeit [a + b for a,b in zip(l1,l2)]
%timeit [l1[i] + l2[i] for i in range(len(l1))]

16 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
23.9 µs ± 589 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
40.1 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
